In [3]:
import numpy as np
import gym
import gym_game
import random

In [4]:
env = gym.make("Pygame-v0", )

## A2C with DND

In [5]:
from model import DNDLSTM as Agent
from model import compute_a2c_loss, compute_returns
import torch
import torch.nn as nn

In [6]:
n_trials = 9999
trial_length = 1000
learning_rate = 5e-4
dim_hidden = 128
# init agent / optimizer
agent = Agent(env.observation_space.shape[0],dim_hidden, env.action_space.n, 5)
optimizer = torch.optim.Adam(agent.parameters(), lr=learning_rate)

In [7]:
def simulate():
    global epsilon, epsilon_decay
    for episode in range(n_trials):

        # Init environment
        state = env.reset()
        probs, rewards, values = [], [], []
        h_t, c_t = agent.get_init_states()

        # AI tries up to MAX_TRY times
        for t in range(trial_length):
            # only save memory at the last time point
            agent.turn_off_encoding()
            if t == trial_length-1:
                agent.turn_on_encoding()

            # A2C agent picks action
            output_t, _ = agent(torch.tensor(state).float().view(1, 1, -1), h_t, c_t) ### .view(1,1,-1) for LSTM input with (seq_len, batch, input_size)
            action, action_prob, value, h_t, c_t = output_t

            # Do action and get result
            next_state, reward, done, _ = env.step(action)
            reward += env.pygame.car.average_speed/env.pygame.car.time_spent * 10  ### encourage the car to go faster
            probs.append(action_prob)
            rewards.append(reward/200)
            values.append(value)

            # Set up for the next iteration, and remember last action
            state = next_state

            # Draw games
            env.render()

            # When episode is done, print reward
            if done or t >= trial_length - 1:
                print("Episode %d finished after %i time steps with total reward = %f." % (episode, t, sum(rewards)))
                break

        # returns = compute_returns(rewards, gamma=0.1, normalize=True) + 0.01
        # loss_policy, loss_value = compute_a2c_loss(probs, values, returns)
        # loss = loss_policy + loss_value
        # print('loss = %f' % (loss.item()))
        # nn.utils.clip_grad_norm_(agent.parameters(), 0.5)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

In [8]:
simulate()

Episode 0 finished after 11 time steps with total reward = -49.475658.
Episode 1 finished after 20 time steps with total reward = -49.300529.
Episode 2 finished after 49 time steps with total reward = -48.489878.
Episode 3 finished after 30 time steps with total reward = -48.987102.
Episode 4 finished after 23 time steps with total reward = -49.152685.
Episode 5 finished after 78 time steps with total reward = -48.615051.
Episode 6 finished after 31 time steps with total reward = -49.055326.
Episode 7 finished after 29 time steps with total reward = -48.657599.
Episode 8 finished after 27 time steps with total reward = -48.120592.
Episode 9 finished after 24 time steps with total reward = -48.775387.
Episode 10 finished after 20 time steps with total reward = -48.526749.
Episode 11 finished after 39 time steps with total reward = -48.897891.
Episode 12 finished after 39 time steps with total reward = -48.641053.
Episode 13 finished after 14 time steps with total reward = -49.453755.
Ep